<a href="https://colab.research.google.com/github/suna0107/ANN_DL101/blob/main/%E2%98%85(250401)3_2_ITEM_HS6_%EC%9E%84%EB%B2%A0%EB%94%A9_%EC%B5%9C%EC%A2%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. ITEM 정보 내 HSKCD

- 목표 : ITEM 정보 중 HSKCD(10자리)에서 HS_CODE_6 (6자리 추출)

  → hs_avg_df: HS 6자리별 벡터 테이블의 hs_vec_800 열과 붙이기

## 1. HS 6자리 기준 코드/국문/영문 데이터 만들기 = hs_info_df

- 학습용 x , 추후 분석용 데이터

- "hs_code6_name.csv"

In [25]:
import pandas as pd

# 1. 원본 HS 코드 데이터 불러오기
hs_full_df = pd.read_excel("/content/HS_CODE_PROCESSED_kr_en_final.xlsx")

# 2. HS_CODE를 문자열 10자리로 통일 (앞자리 0 보존)
hs_full_df['HS_CODE'] = hs_full_df['HS_CODE'].astype(str).str.zfill(10)

# 3. 앞 6자리 추출
hs_full_df['HS_CODE_6'] = hs_full_df['HS_CODE'].str[:6]

FileNotFoundError: [Errno 2] No such file or directory: '/content/HS_CODE_PROCESSED_kr_en_final.xlsx'

In [ ]:
# 4. 뒤 4자리가 '0000'인 경우만 필터링 (세부 품목 제외)
hs_6digit_only = hs_full_df[hs_full_df['HS_CODE'].str[6:] == '0000']

# 5. HS_CODE_6 기준으로 첫 번째 행만 대표로 선택 (국문/영문명)
hs_info_df = hs_6digit_only.groupby('HS_CODE_6').first().reset_index()


In [ ]:
# 6. 필요한 컬럼만 추출 및 이름 정리
hs_info_df = hs_info_df[['HS_CODE_6', '국문', '영문']]
hs_info_df.columns = ['HS_CODE_6', 'HS_NAME_KR', 'HS_NAME_EN']

In [ ]:
hs_info_df

In [ ]:
# 7. 저장
hs_info_df.to_csv("hs_code6_name.csv", index=False, encoding="utf-8-sig")      # CSV 저장
hs_info_df.to_excel("hs_code6_name.xlsx", index=False)

##2. ITEM 중 필요한 데이터 정리 : item_data.csv

- 이용할 컬럼 : 물품명, 모델명, 규격, 분류, 회사명, HSKCD, 결과

- 결과가 있는 행: 학습용

- 결과가 없는 행: 예측 대상용 (입력 준비용)



In [60]:
import pandas as pd

# 1. 엑셀 파일에서 ITEM 데이터 불러오기
item_df = pd.read_excel("/content/ITEM.xlsx")

# 2. 필요한 컬럼만 추출
columns = ['물품명', '모델명', '규격', '분야', '회사명', 'HSKCD', '결과', '통제번호']
item_selected = item_df[columns]

# 3. 전체 ITEM 데이터 저장 (CSV & Excel)
item_selected.to_csv("item_data.csv", index=False, encoding="utf-8-sig")
item_selected.to_excel("item_data.xlsx", index=False)


# 3. ITEM 정보의 SBERT 입력 벡터 만들기

(1) HSKCD 앞 6자리 추출 → HS_CODE_6
- HS_CODE_6과 HS정보들 연결하기

- hs_avg_df → HS 6자리별 평균 벡터 hs_vec_800 가져오기 : 실제 학습용 데이터
- hs_info_df → HS 6자리별 이름(국문/영문) 가져오기 : 분석용 데이터

(2) SBERT로 임베딩

1) 물품명 + 모델명 + 규격 : item_vec(384)

2) 회사명 : 회사_vec(384)

* 회사명을 따로 SBERT 임베딩하는 이유?
실제로 회사별로 자주 쓰는 HS 코드나 품목이 반복되면, 모델이 "이 회사는 보통 이런 거 판정받는다"는 패턴을 학습할 수 있음.

(3) item_vec(768) ,  회사명_vec(384), hs_vec_800(800) concat → 모델 입력

- concat을 위해서는 행의 수, 차원(열)이 동일 해야함

- 특이사항 : 정보보안, 소프트웨어 분야의 경우 HSKCD 없는 경우 존재함
-> 이런 경우, 추후 CONCAT을 위해 NaN 대신에 0으로 패딩하여 대체필요



###1. HSKCD에서 6자리 정보 추출 후 평균벡터 붙이기

In [62]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

# 1. 데이터 불러오기
item_df = pd.read_csv("/content/item_data.csv")   # 또는 item_test.csv
hs_avg_df = pd.read_pickle("/content/hs_code6_avg_vec_800d.pkl")

# 2. HS 6자리 추출
item_df['HS_CODE_6'] = item_df['HSKCD'].astype(str).str[:6]

# 3. HS 평균 벡터 붙이기
item_df = item_df.merge(hs_avg_df[['HS_CODE_6', 'hs_vec_800']], on='HS_CODE_6', how='left')


In [63]:
item_df.head(3)

,물품명,모델명,규격,분야,회사명,HSKCD,결과,통제번호,HS_CODE_6,hs_vec_800
0,위상 노이즈 분석기&VCO 테스터(Phase noise Analyzer and VC...,R&S® FSWP50,Frequency : 1MHz to 50GHz/ Option : B1 B4 B24 ...,전자,주식회사 딥웨이브,9030400000,비해당,3A002.c.,903040,"[-0.036086276173591614, 0.31613555550575256, -..."
1,PCB ASSY,RJ154-0218,ARC PCB ASSY 규격:245x124x36 무게:0.544kg 재질:FR-4 ...,전자,(주)현대에버다임,8534009000,비해당,NaN,853400,"[-0.06912247091531754, 0.220761239528656, -0.1..."
2,REGULATOR,RA450-0019A,11-002-061(큰),건설 및 운송기계,(주)현대에버다임,7326909000,비해당,NaN,732690,"[-0.0746660903096199, 0.2519918978214264, -0.1..."


###2. [item_vec] 물품명 + 모델명 + 규격 + 분야→ SBERT

In [64]:
# SBERT 모델 로드
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# 텍스트 조합 (물품명 + 모델명 + 규격 + 분류)
item_df['item_text'] = item_df.apply(
    lambda row: f"물품명: {row['물품명']} 모델명: {row['모델명']} 규격: {row['규격']} 분야: {row['분야']}",
    axis=1
)


In [78]:
item_df['item_text'].tail(1)

,item_text
2864,물품명: Cisco 10GBASE SFP+ Modules 모델명: SFP-H10GB...


- sbert는 토큰 길이가 512 초과하면 성능 떨어짐 -> 확인 필요

In [79]:
from transformers import AutoTokenizer

# 1. 토크나이저 불러오기 (SBERT 모델과 맞춰야 함)
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# 2. 토큰 길이 리스트만 계산 (item_df에는 저장 안 함)
token_lens = item_df['item_text'].apply(lambda x: len(tokenizer.tokenize(x)))

# 3. 512 초과한 행만 필터링해서 확인 (원본 그대로 출력됨)
long_texts = item_df[token_lens > 512]

# 4. 결과 출력 : 8개라서 그냥 무시하고 진행하기
print("🔍 512 토큰 초과한 행 수:", len(long_texts))
print(long_texts[['item_text']].head())


Token indices sequence length is longer than the specified maximum sequence length for this model (538 > 512). Running this sequence through the model will result in indexing errors


🔍 512 토큰 초과한 행 수: 8
                                              item_text
191   물품명: LEVEL GAUGE 모델명: 10-LG-10237 / 10-LG-1025...
357   물품명: BOLT & NUTS - B8 / B8M 모델명: B8-1, B8-2, B...
692   물품명: STRAINER 모델명: TAG NO : SPS-2811C/SPS-2811...
693   물품명: FILTER  모델명: TAG NO : SPS-2811C/SPS-2811A...
1897  물품명: STRAINER(VALVE류 제외) 모델명: TAG NO: 2SR-5521...


In [66]:
from tqdm import tqdm

# SBERT 임베딩
item_texts = item_df['item_text'].astype(str).tolist()
item_vecs = sbert.encode(item_texts)
# 진행상황 확인
item_vecs = [sbert.encode(text) for text in tqdm(item_texts)]

100%|██████████| 2865/2865 [00:34<00:00, 82.83it/s]


In [67]:
# item 텍스트 임베딩 결과를 리스트화 하여 'item_df'열에 저장
item_df['item_vec'] = list(item_vecs)

In [68]:
item_df['item_vec']

,item_vec
0,"[-0.13495076, -0.16238776, -0.14617763, -0.568..."
1,"[0.35064232, -0.3505638, -0.6376528, -0.041862..."
2,"[-0.025563722, 0.13539273, -0.26119855, 0.8144..."
3,"[0.02439301, -0.45042038, -0.5502303, 0.137376..."
4,"[0.23377459, 0.20144418, -0.6715343, 0.3352779..."
...,...
2860,"[-0.07856645, 0.09103005, -0.28533533, -0.4154..."
2861,"[0.16751932, 0.14041051, -0.5645307, 0.3581503..."
2862,"[0.1505563, -0.36786667, 0.123948716, -0.57130..."
2863,"[0.22787055, -0.20284691, 0.06262538, 0.165827..."


### 3.  [회사_vec] 회사명만 따로 SBERT

In [69]:
company_name = item_df['회사명'].astype(str).tolist()
company_vecs = sbert.encode(company_name)

item_df['company_vecs'] = list(company_vecs)

- item_df 내 임베딩 데이터 모두 저장한 파일 = item_vec.pkl

In [47]:

# 1. 피클 파일로 저장 (SBERT 벡터는 리스트 → 피클 권장)
item_df.to_pickle("item_df_vec.pkl")

# 2. csv 저장
item_df.to_csv("item_df_vec.csv", index=False, encoding="utf-8-sig")


### 4. 세 벡터 concat → 최종 input_vec (1568차원)

(1) concat을 위해서 행과 차원이 일치해야함

- 소프트웨어, 정보보안 등 일부 HSKCD 없이도 판정 가능
- 추후, 모델 입력값은 차원, 행이 일치해함 -> 따라서, HS코드가 누락인 경우 NAN이 아닌 0으로 패딩해서 차원을 동일하게 맞추기

In [70]:
print("전체 데이터프레임 행 수:", len(item_df))
print("item_vec 있는 행 수:", item_df['item_vec'].notnull().sum())
print("회사_vec 있는 행 수:", item_df['company_vecs'].notnull().sum())
print("hs_vec_800 있는 행 수:", item_df['hs_vec_800'].notnull().sum())

전체 데이터프레임 행 수: 2865
item_vec 있는 행 수: 2865
회사_vec 있는 행 수: 2865
hs_vec_800 있는 행 수: 2803


In [71]:
#  누락된 행 확인 : 정보보안, 소프트웨어의 경우 HSKCD없이도 판정 가능
# 추후 CONCAT을 위해 NAN이 아닌 0으로 패딩하여 모든 차원, 행의 수 맞추기
item_df[item_df['hs_vec_800'].isnull()]


,물품명,모델명,규격,분야,회사명,HSKCD,결과,통제번호,HS_CODE_6,hs_vec_800,item_text,item_vec,company_vecs
143,ixShield,ixShield v1.5,ixShield v1.5,정보보안,(주)엔에스에이치씨,-,해당,5D002.c.1.,-,NaN,물품명: ixShield 모델명: ixShield v1.5 규격: ixShield...,"[-0.83767056, -0.43451333, -0.32771742, -0.010...","[-0.8262312, -0.37119642, -0.3940577, 0.014397..."
182,DevExpress Universal 24.2,DevExpress Universal 24.2,소프트웨어 개발 패키지 툴,컴퓨터,단군소프트,-,비해당,NaN,-,NaN,물품명: DevExpress Universal 24.2 모델명: DevExpress...,"[-0.6397082, -0.22082311, 0.22587995, -0.20364...","[-0.22656195, -0.8173312, 0.07579139, 0.274589..."
241,aStone,aStone,무형 소프트웨어로서 별도의 규격은 없습니다.,컴퓨터,주식회사 데이타솔루션,-,해당,5D002.c.1.,-,NaN,물품명: aStone 모델명: aStone 규격: 무형 소프트웨어로서 별도의 규격은...,"[-0.69128555, 0.68512493, -0.34471792, 0.07014...","[-0.7393956, 0.1982225, -1.4073247, -0.0276085..."
246,해외(페루) 현지조선소 교육자료 - (Understanding of Block Fa...,페루 호위함/원해경비함/상륙함,함정 건조(생산) 외에 응용분야 없음,함정,에이치디현대중공업 주식회사,-,비해당,ML22.,-,NaN,물품명: 해외(페루) 현지조선소 교육자료 - (Understanding of Blo...,"[-0.23282391, 0.15263645, -0.81129146, 0.20316...","[-1.1808014, -0.1531619, -0.5656478, 0.0795114..."
247,해외(페루) 현지조선소 교육자료 - Welding Process include ta...,페루 호위함/원해경비함/상륙함,함정 건조(생산) 외에 응용분야 없음,함정,에이치디현대중공업 주식회사,-,비해당,ML22.,-,NaN,물품명: 해외(페루) 현지조선소 교육자료 - Welding Process inclu...,"[-0.33357966, 0.14811441, -0.8402546, 0.253376...","[-1.1808014, -0.1531619, -0.5656478, 0.0795114..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260,Research Proposal Advancing Li-ion Battery Pac...,해당없음,이차전지 모듈팩 생산·사용,전자,KAIST,-,비해당,NaN,-,NaN,물품명: Research Proposal Advancing Li-ion Batter...,"[-0.03236578, -0.051285308, -0.40731376, -0.37...","[0.053614974, 0.6122125, 0.1804245, -0.9448639..."
2374,보툴리눔 독소 제제 생산기술,"ATGC-100(BOTOX), ATGC-110((XEOMIN)",보툴리눔 독소는 신경전달물질(아세틸콜린)의 분비를 차단하여 근육의 수축을 막는 단순...,생물,(주) 에이티지씨,-,해당,1E001.,-,NaN,"물품명: 보툴리눔 독소 제제 생산기술 모델명: ATGC-100(BOTOX), ATG...","[0.06037862, -0.25577, -0.30189443, 0.23524892...","[-0.88395643, 0.41650215, -1.6644671, -0.06898..."
2389,소프트웨어,RADMAX,무형의 소프트웨어,정보보안,디알젬,-,비해당,5D002.c.1.,-,NaN,물품명: 소프트웨어 모델명: RADMAX 규격: 무형의 소프트웨어 분야: 정보보안,"[-0.29601827, 0.5894548, 0.045839366, 0.137013...","[-1.2612818, -0.2509368, -0.77734786, -1.15135..."
2411,Soft x-ray Charger,XRC-05,알루미늄,전자,(주)에이치시티엠,-,비해당,NaN,-,NaN,물품명: Soft x-ray Charger 모델명: XRC-05 규격: 알루미늄 분...,"[0.21989256, -0.26572984, -0.48946342, -0.0187...","[-0.9223122, -0.16033792, -0.65209955, 0.66986..."


(2) concat 하여 최종 input_vec 만들기 (없으면 padding)

In [72]:
# 11. 벡터 concat 후 padding (최종 input_vec = 1568차원)

def build_input_vec(row):
    base_vec = np.concatenate([row['item_vec'], row['company_vecs']])  # 384 + 384 = 768
    if isinstance(row['hs_vec_800'], (list, np.ndarray)):
        return np.concatenate([base_vec, row['hs_vec_800']])  # 총 1568
    else:
        return np.concatenate([base_vec, np.zeros(800)])      # HS 없으면 → 0-padding하여 1568차원 만들기

item_df['input_vec'] = item_df.apply(build_input_vec, axis=1)


In [73]:
# 결과 확인
print("✅ input_vec 차원:", len(item_df['input_vec'].iloc[0]))  # 항상 1568
print("전체 행 수:", len(item_df))
print("HS 누락된 행 수:", item_df['hs_vec_800'].isnull().sum())

✅ input_vec 차원: 2336
전체 행 수: 2865
HS 누락된 행 수: 62


In [74]:
print("item_vec 차원:", len(item_df['item_vec'].iloc[0]))
print("company_vecs 차원:", len(item_df['company_vecs'].iloc[0]))
print("hs_vec_800 차원:", len(item_df['hs_vec_800'].dropna().iloc[0]))  # 있는 값 중 하나


item_vec 차원: 768
company_vecs 차원: 768
hs_vec_800 차원: 800


In [75]:
# 1. 최종 모델 입력데이터 = item_input.pkl
item_df.to_pickle("item_input.pkl")
#  2. csv 저장
item_df.to_csv("item_input.csv", index=False, encoding="utf-8-sig")